---
## SQL Agents
Fourth approach, from *create_sql_agent*, contains *SQLDatabaseToolkit*
* It can answer questions based on the databases' schema as well as on the databases' content (like describing a specific table)
* It can recover from errors by running a generated query, catching the traceback and regenerating it correctly


In [ ]:
from langchain.utilities import SQLDatabase
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
# from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.llms import GPT4All
from langchain.agents import AgentExecutor


db = SQLDatabase.from_uri("sqlite:///../db/retention-sqlite.db")
llm = GPT4All(model_name="../../", verbose=True)

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)